In [1]:
import numpy as np
import pandas as pd


def invert_T(T):
    R = T[:3, :3]
    t = T[:3, 3]
    Ti = np.eye(4)
    Ti[:3, :3] = R.T
    Ti[:3, 3] = -R.T @ t
    return Ti

def compose(R, t):
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    return T

def euler_ZYX_to_R(z, y, x):
    cz, sz = np.cos(z), np.sin(z)
    cy, sy = np.cos(y), np.sin(y)
    cx, sx = np.cos(x), np.sin(x)

    Rz = np.array([[ cz, -sz,  0],
                   [ sz,  cz,  0],
                   [  0,   0,  1]])

    Ry = np.array([[ cy,  0, sy],
                   [  0,  1,  0],
                   [-sy,  0, cy]])

    Rx = np.array([[ 1,  0,   0],
                   [ 0, cx, -sx],
                   [ 0, sx,  cx]])

    return Rz @ Ry @ Rx

def log_SO3(R):
    trc = np.clip((np.trace(R) - 1) / 2.0, -1.0, 1.0)
    theta = np.arccos(trc)
    if theta < 1e-12:
        return np.zeros(3)
    w = np.array([R[2,1] - R[1,2],
                  R[0,2] - R[2,0],
                  R[1,0] - R[0,1]]) / (2 * np.sin(theta))
    return theta * w

In [ ]:
def load_poses_csv(path):
    df = pd.read_csv(
        path,
        sep=r'(?:,\s+|\s+)',
        usecols=range(7),
        header=None,
        engine="python",
        names=["id", "X", "Y", "Z", "RZ", "RY", "RX"]
    )
    return df

def df_to_Ts(df):
    Ts = []
    for _, r in df.iterrows():
        R = euler_ZYX_to_R(
            np.deg2rad(r["RZ"]),
            np.deg2rad(r["RY"]),
            np.deg2rad(r["RX"])
        )
        t = np.array([r["X"], r["Y"], r["Z"]])
        Ts.append(compose(R, t))
    return np.array(Ts)

In [ ]:
def summarize_errors(As, Bs, X, Y):
    t_errs = []
    r_errs = []
    for A, B in zip(As, Bs):
        Delta = invert_T(A @ X) @ (Y @ B)
        t_errs.append(np.linalg.norm(Delta[:3, 3]))
        angle = np.linalg.norm(log_SO3(Delta[:3, :3])) * 180.0 / np.pi
        r_errs.append(angle)

    t_errs = np.array(t_errs)
    r_errs = np.array(r_errs)

    def stats(x):
        return {
            "mean": float(np.mean(x)),
            "median": float(np.median(x)),
            "rmse": float(np.sqrt(np.mean(x**2))),
            "p95": float(np.percentile(x, 95)),
            "max": float(np.max(x))
        }
    return stats(t_errs), stats(r_errs)

In [4]:
# https://tsapps.nist.gov/publication/get_pdf.cfm?pub_id=910225
# https://math.loyola.edu/~mili/Calibration/AXYB/shah.m

def shah(As, Bs):
    n = len(As)

    T9 = np.zeros((9, 9))
    for i in range(n):
        Ra = As[i][:3, :3]
        Rb = Bs[i][:3, :3]
        T9 = T9 + np.kron(Rb, Ra)

    U, S, Vt = np.linalg.svd(T9)
    x = Vt.T[:, 0]
    y = U[:, 0]

    Xr = np.reshape(x, (3, 3), order='F')
    detX = np.linalg.det(Xr)
    Xr = (np.sign(detX) / (np.abs(detX) ** (1/3))) * Xr
    Ux, Sx, Vtx = np.linalg.svd(Xr)
    Xr = Ux @ Vtx

    Yr = np.reshape(y, (3, 3), order='F')
    detY = np.linalg.det(Yr)
    Yr = (np.sign(detY) / (np.abs(detY) ** (1/3))) * Yr
    Uy, Sy, Vty = np.linalg.svd(Yr)
    Yr = Uy @ Vty

    A_lin = np.zeros((3 * n, 6))
    b_lin = np.zeros((3 * n, 1))

    vecY = np.reshape(Yr, (9, 1), order='F')

    for i in range(n):
        Ra = As[i][:3, :3]
        ta = As[i][:3, 3].reshape(3, 1)
        tb = Bs[i][:3, 3].reshape(3, 1)

        A_lin[3*i:3*i+3, 0:3] = -Ra
        A_lin[3*i:3*i+3, 3:6] = np.eye(3)

        K = np.kron(tb.T, np.eye(3))
        b_lin[3*i:3*i+3, 0:1] = ta - K @ vecY

    t_sol, _, _, _ = np.linalg.lstsq(A_lin, b_lin, rcond=None)
    tX = t_sol[0:3].reshape(3)
    tY = t_sol[3:6].reshape(3)

    X = compose(Xr, tX)
    Y = compose(Yr, tY)
    return X, Y

In [5]:
np.set_printoptions(precision=6, suppress=True)

file_A = "./data/bigcalib/MeasuredPositions.txt"
file_B = "./data/bigcalib/RobotPositions.txt"

dfA = load_poses_csv(file_A)
dfB = load_poses_csv(file_B)

As = df_to_Ts(dfA)
Bs = df_to_Ts(dfB)

X, Y = shah(As, Bs)

print("X = ")
print(X)
print("Y = ")
print(Y)

Xinv = invert_T(X)
Yinv = invert_T(Y)
print("X^{-1} = ")
print(Xinv)
print("Y^{-1} = ")
print(Yinv)

t_stats, r_stats = summarize_errors(As, Bs, X, Y)
print("Ошибки (translation, mm):", t_stats)
print("Ошибки (rotation, deg): ", r_stats)

X = 
[[ 0.99999  -0.002189  0.003954  0.47651 ]
 [ 0.002186  0.999997  0.000646  3.809908]
 [-0.003956 -0.000638  0.999992 77.494311]
 [ 0.        0.        0.        1.      ]]
Y = 
[[    0.99994     -0.010143     0.004218 -6265.219803]
 [    0.010201     0.999852    -0.013884  -181.343607]
 [   -0.004076     0.013926     0.999895  3122.592561]
 [    0.           0.           0.           1.      ]]
X^{-1} = 
[[  0.99999    0.002186  -0.003956  -0.178296]
 [ -0.002189   0.999997  -0.000638  -3.759437]
 [  0.003954   0.000646   0.999992 -77.498036]
 [  0.         0.         0.         1.      ]]
Y^{-1} = 
[[    0.99994      0.010201    -0.004076  6279.419773]
 [   -0.010143     0.999852     0.013926    74.281555]
 [    0.004218    -0.013884     0.999895 -3098.357398]
 [    0.           0.           0.           1.      ]]
Ошибки (translation, mm): {'mean': 3.075945229584242, 'median': 2.92247563821688, 'rmse': 4.008710257661779, 'p95': 4.790495091232605, 'max': 60.96480601065306}
Ошибк